In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.bidir_lstm_multihead import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/bidir_lstm/10k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/bidir_lstm/10k', 'model': {'type': 'Bidirectional Multihead LSTM', 'lstm_units': 128, 'num_lstm_layers': 2, 'attention': True}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/10k_single_agent.csv'}}


In [6]:
model_params = {
    'lstm_units': config['model']['lstm_units'],
    'num_lstm_layers': config['model']['num_lstm_layers'],
    'attention': config['model']['attention']
}

In [7]:
# Load and preprocess data
df = load_data(f"../../../../data/processed/{config['data']['path']}")
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model,
)

Epoch 1/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 50s 38ms/step - loss: 2.9698 - output_1_accuracy: 0.8944 - output_2_accuracy: 0.7141 - output_3_accuracy: 0.2099 - val_loss: 2.2507 - val_output_1_accuracy: 0.9166 - val_output_2_accuracy: 0.9311 - val_output_3_accuracy: 0.2608
Epoch 2/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 47s 39ms/step - loss: 2.2115 - output_1_accuracy: 0.9340 - output_2_accuracy: 0.9335 - output_3_accuracy: 0.2664 - val_loss: 2.1604 - val_output_1_accuracy: 0.9358 - val_output_2_accuracy: 0.9357 - val_output_3_accuracy: 0.2664
Epoch 3/5
 155/1207 ━━━━━━━━━━━━━━━━━━━━ 37s 36ms/step - loss: 2.1747 - output_1_accuracy: 0.9281 - output_2_accuracy: 0.9281 - output_3_accuracy: 0.2749

KeyboardInterrupt: 

In [ ]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

In [ ]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f"../../../../models/{config['name']}.h5",
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)